In [3]:
import torch
from tqdm import tqdm
from torch import nn, optim
from torch.nn import init
import pandas as pd
import pandas as pd
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

data = pd.read_csv("../../data2/data.csv").drop(columns = ["id", "source", 
                                                       "latitude", "longitude"])

In [11]:
class TabularFFNNSimple(nn.Module):
    def __init__(self, input_size, output_size, dropout_prob=0.4):
        super(TabularFFNNSimple, self).__init__()
        hidden_size = 64
        self.ffnn = nn.Sequential(
            nn.Linear(input_size, hidden_size),
            nn.ReLU(),
            nn.BatchNorm1d(hidden_size),
            nn.Dropout(0.25),
            nn.Linear(hidden_size, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, hidden_size),
            nn.ReLU(),
            nn.Dropout(0.18),
            nn.Linear(hidden_size, output_size)
        )
        
        for m in self.ffnn:
            if isinstance(m, nn.Linear):
                init.xavier_uniform_(m.weight)
                m.bias.data.fill_(0)

    def forward(self, x):
        x = x.float()
        # print(x)
        x = x.view(x.size(0), -1)
        x = self.ffnn(x)
        return x
    
# Split the data into features and target
X = data.drop('price', axis=1)
y = data['price']

# Standardize the features
device = torch.device("cpu")
# Convert to PyTorch tensors
X_tensor = torch.tensor(X.to_numpy(), dtype=torch.float32, device = device)
y_tensor = torch.tensor(y.values, dtype=torch.float32, device = device)


# Split the data into training and combined validation and testing sets
X_train, X_val_test, y_train, y_val_test = train_test_split(X_tensor, y_tensor,
                                                            test_size=0.4, random_state=42)

# Split the combined validation and testing sets
X_val, X_test, y_val, y_test = train_test_split(X_val_test, y_val_test, test_size=0.5, random_state=42)

# Create DataLoader for training, validation, and testing
train_data = TensorDataset(X_train, y_train)
val_data = TensorDataset(X_val, y_val)
test_data = TensorDataset(X_test, y_test)

batch_size = 256
train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_data, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=False)

# Check if the dimensions match the expected input size for the model
input_size = X_train.shape[1]

# Output
# input_size, train_loader, test_loader

model = TabularFFNNSimple(
    input_size = input_size,
    output_size = 1
)
model.to(device)

num_epochs = 300000
train_losses = []
val_losses = []
epochs_suc = [] # to have a reference to it
grad_norms = []

def get_gradient_norm(model):
    total_norm = 0
    for p in model.parameters():
        if p.grad is not None:
            param_norm = p.grad.data.norm(2)
            total_norm += param_norm.item() ** 2
    total_norm = total_norm ** 0.5
    return total_norm

In [ ]:
optimizer = optim.Adam(
    model.parameters(), 
    lr=1e-2,
    weight_decay=3e-8
)
criterion = torch.nn.MSELoss()
criterion_abs = torch.nn.L1Loss()
criterion = criterion_abs

scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
    optimizer, 
    mode='min', 
    factor=0.99, 
    patience=10, 
    verbose=True
)

for epoch in range(num_epochs):
    # Training
    model.train()  # Set the model to training mode
    running_loss = 0.0
    l1_losses = []
    grad_norm = 0
    for tuple_ in train_loader:
        datas, prices = tuple_
        optimizer.zero_grad()
        outputs = model(datas)
        prices_viewed = prices.view(-1, 1).float()
        loss = criterion(outputs, prices_viewed)
        loss.backward()
        grad_norm += get_gradient_norm(model)
        optimizer.step()

        running_loss += loss.item()
        
    grad_norms.append(grad_norm / len(train_loader))
    train_losses.append(running_loss / len(train_loader))  # Average loss for this epoch

    # Validation
    model.eval()  # Set the model to evaluation mode
    val_loss = 0.0
    with torch.no_grad():  # Disable gradient calculation
        for tuple_ in val_loader:
            datas, prices = tuple_
            outputs = model(datas)  # Forward pass
            prices_viewed = prices.view(-1, 1).float()
            loss = criterion(outputs, prices_viewed)  # Compute loss
            val_loss += loss.item()  # Accumulate the loss
            l1_losses.append(criterion_abs(outputs, prices_viewed))

    val_losses.append(val_loss / len(val_loader))  # Average loss for this epoch
    l1_mean_loss = sum(l1_losses) / len(l1_losses)
    # Print epoch's summary
    epochs_suc.append(epoch)
#     scheduler.step(val_losses[-1])
    if epoch % 100 == 0:
        tl = f"Training Loss: {int(train_losses[-1])}"
        vl = f"Validation Loss: {int(val_losses[-1])}"
        l1 = f"L1: {int(l1_mean_loss)}"
        dl = f'Epoch {epoch+1}, {tl}, {vl}, {grad_norms[-1]}'
        print(dl)

Epoch 1, Training Loss: 238861, Validation Loss: 244200, 4.214949734277446
Epoch 101, Training Loss: 90427, Validation Loss: 85212, 46591.10569567181
Epoch 201, Training Loss: 60667, Validation Loss: 59709, 46996.231576686136
Epoch 301, Training Loss: 56532, Validation Loss: 57938, 105827.72296845086
Epoch 401, Training Loss: 58469, Validation Loss: 109079, 81073.29031014674
Epoch 501, Training Loss: 56858, Validation Loss: 87463, 144752.91718681835
Epoch 601, Training Loss: 57392, Validation Loss: 63296, 119062.88803647253
Epoch 701, Training Loss: 53282, Validation Loss: 54177, 91077.69759589313
Epoch 801, Training Loss: 53872, Validation Loss: 55014, 137481.90727323657
Epoch 901, Training Loss: 54227, Validation Loss: 59996, 89935.77397105248
Epoch 1001, Training Loss: 57951, Validation Loss: 64338, 145172.02888849788
Epoch 1101, Training Loss: 55216, Validation Loss: 61281, 112962.9381357003
Epoch 1201, Training Loss: 55150, Validation Loss: 60853, 131534.3590832935
Epoch 1301, Tra

Epoch 10801, Training Loss: 53950, Validation Loss: 54552, 73535.5846425256
Epoch 10901, Training Loss: 48175, Validation Loss: 60518, 82441.39427669447
Epoch 11001, Training Loss: 46779, Validation Loss: 53371, 84815.93888810977
Epoch 11101, Training Loss: 50508, Validation Loss: 52406, 75027.76473942808
Epoch 11201, Training Loss: 47601, Validation Loss: 56850, 64498.02762344821
Epoch 11301, Training Loss: 48320, Validation Loss: 61954, 85143.28298273365
Epoch 11401, Training Loss: 48949, Validation Loss: 52691, 67554.34392519917
Epoch 11501, Training Loss: 52233, Validation Loss: 61394, 63089.2203073933
Epoch 11601, Training Loss: 50126, Validation Loss: 53804, 100870.64150300702
Epoch 11701, Training Loss: 50776, Validation Loss: 50905, 86583.78371960456
Epoch 11801, Training Loss: 47050, Validation Loss: 52312, 93723.03717988478
Epoch 11901, Training Loss: 52211, Validation Loss: 51667, 72024.91314335592
Epoch 12001, Training Loss: 48372, Validation Loss: 52611, 77963.11979596289


Epoch 21501, Training Loss: 47545, Validation Loss: 56397, 77519.89410798183
Epoch 21601, Training Loss: 47079, Validation Loss: 54467, 59418.46822398193
Epoch 21701, Training Loss: 46006, Validation Loss: 51961, 59526.32420385396
Epoch 21801, Training Loss: 49740, Validation Loss: 50325, 93111.53577954204
Epoch 21901, Training Loss: 46980, Validation Loss: 54481, 92065.45365493304
Epoch 22001, Training Loss: 47486, Validation Loss: 52304, 56152.45149734978
Epoch 22101, Training Loss: 48692, Validation Loss: 54498, 74236.20759611648
Epoch 22201, Training Loss: 47513, Validation Loss: 54029, 93820.37526371462
Epoch 22301, Training Loss: 47761, Validation Loss: 52159, 67517.65985808753
Epoch 22401, Training Loss: 48467, Validation Loss: 54419, 56869.8398675553
Epoch 22501, Training Loss: 45424, Validation Loss: 60403, 54406.92254835556
Epoch 22601, Training Loss: 47762, Validation Loss: 53233, 63435.264890397106
Epoch 22701, Training Loss: 46663, Validation Loss: 50679, 58960.67003682267

Epoch 32201, Training Loss: 47059, Validation Loss: 55772, 62176.49963507443
Epoch 32301, Training Loss: 47504, Validation Loss: 53291, 55893.4272145222
Epoch 32401, Training Loss: 48428, Validation Loss: 52518, 40498.11438488892
Epoch 32501, Training Loss: 46342, Validation Loss: 54907, 87504.95021862234
Epoch 32601, Training Loss: 45408, Validation Loss: 54585, 71652.45792224759
Epoch 32701, Training Loss: 45696, Validation Loss: 62242, 53158.46189491507
Epoch 32801, Training Loss: 46574, Validation Loss: 52939, 58676.24763973346
Epoch 32901, Training Loss: 48576, Validation Loss: 52886, 57766.413240128524
Epoch 33001, Training Loss: 46436, Validation Loss: 62546, 54002.462141756325
Epoch 33101, Training Loss: 46710, Validation Loss: 54157, 73184.19536276972
Epoch 33201, Training Loss: 43990, Validation Loss: 52847, 52222.91568651723
Epoch 33301, Training Loss: 48840, Validation Loss: 64906, 62491.771787636506
Epoch 33401, Training Loss: 46707, Validation Loss: 54462, 47971.399512476

Epoch 42901, Training Loss: 42810, Validation Loss: 54395, 43905.831042026424
Epoch 43001, Training Loss: 44964, Validation Loss: 54609, 38948.17465065432
Epoch 43101, Training Loss: 44277, Validation Loss: 54416, 44764.64815862203
Epoch 43201, Training Loss: 48486, Validation Loss: 60925, 55002.20301259755
Epoch 43301, Training Loss: 47632, Validation Loss: 52278, 48988.98963199052
Epoch 43401, Training Loss: 46718, Validation Loss: 51149, 52431.1464741811
Epoch 43501, Training Loss: 43699, Validation Loss: 53959, 49066.60540942687
Epoch 43601, Training Loss: 45216, Validation Loss: 56192, 55053.401717279405
Epoch 43701, Training Loss: 43001, Validation Loss: 57556, 46319.62167806844
Epoch 43801, Training Loss: 45137, Validation Loss: 51297, 47428.41185127065
Epoch 43901, Training Loss: 45460, Validation Loss: 61944, 56695.01644801701
Epoch 44001, Training Loss: 44594, Validation Loss: 51832, 60154.51255115474
Epoch 44101, Training Loss: 45383, Validation Loss: 57854, 60179.1731614728

...

Epoch 85701, Training Loss: 46963, Validation Loss: 53631, 65032.18620253566
Epoch 85801, Training Loss: 46580, Validation Loss: 53408, 42550.690511716864
Epoch 85901, Training Loss: 46049, Validation Loss: 53865, 34268.085770359445
Epoch 86001, Training Loss: 45312, Validation Loss: 53672, 46078.55300031189
Epoch 86101, Training Loss: 43411, Validation Loss: 54053, 35277.04534779417
Epoch 86201, Training Loss: 43946, Validation Loss: 56479, 32370.88120080483
Epoch 86301, Training Loss: 47840, Validation Loss: 58567, 54229.2220488566
Epoch 86401, Training Loss: 48352, Validation Loss: 56942, 37740.24763676989
Epoch 86501, Training Loss: 46378, Validation Loss: 53608, 43272.790645081484
Epoch 86601, Training Loss: 45279, Validation Loss: 55816, 42740.23181009401
Epoch 86701, Training Loss: 44422, Validation Loss: 53549, 37123.04969937962
Epoch 86801, Training Loss: 42811, Validation Loss: 53512, 40271.58937414266
Epoch 86901, Training Loss: 44216, Validation Loss: 53510, 50023.769706706

Epoch 96401, Training Loss: 47323, Validation Loss: 52864, 34842.56525882699
Epoch 96501, Training Loss: 44560, Validation Loss: 55234, 34485.14219965532
Epoch 96601, Training Loss: 45880, Validation Loss: 55954, 38620.226262032505
Epoch 96701, Training Loss: 43128, Validation Loss: 56548, 34335.256966822024
Epoch 96801, Training Loss: 45657, Validation Loss: 57541, 49295.23741113462
Epoch 96901, Training Loss: 45567, Validation Loss: 57142, 45188.14102685976
Epoch 97001, Training Loss: 45064, Validation Loss: 54637, 39386.516861600045
Epoch 97101, Training Loss: 43742, Validation Loss: 52193, 30632.764874367112
Epoch 97201, Training Loss: 43341, Validation Loss: 54203, 49195.180973462724
Epoch 97301, Training Loss: 44448, Validation Loss: 53932, 31085.73036999248
Epoch 97401, Training Loss: 46538, Validation Loss: 53208, 50224.303530200465
Epoch 97501, Training Loss: 44588, Validation Loss: 52582, 45159.74301348402
Epoch 97601, Training Loss: 46404, Validation Loss: 55503, 51387.32937

Epoch 107001, Training Loss: 43654, Validation Loss: 53506, 42807.04522488196
Epoch 107101, Training Loss: 41472, Validation Loss: 56245, 38059.835069681874
Epoch 107201, Training Loss: 39771, Validation Loss: 53772, 32097.12640359187
Epoch 107301, Training Loss: 45927, Validation Loss: 60501, 39205.363681694515
Epoch 107401, Training Loss: 44804, Validation Loss: 57923, 38858.582411653806
Epoch 107501, Training Loss: 43259, Validation Loss: 52537, 42747.205204064354
Epoch 107601, Training Loss: 43845, Validation Loss: 53223, 33010.96968423016
Epoch 107701, Training Loss: 45551, Validation Loss: 54739, 42395.54122551395
Epoch 107801, Training Loss: 44044, Validation Loss: 53184, 43737.67979046488
Epoch 107901, Training Loss: 43406, Validation Loss: 51886, 71047.45003020715
Epoch 108001, Training Loss: 46495, Validation Loss: 58772, 41950.7527870126
Epoch 108101, Training Loss: 42174, Validation Loss: 54088, 37584.01520820079
Epoch 108201, Training Loss: 44909, Validation Loss: 52253, 3

Epoch 117501, Training Loss: 43004, Validation Loss: 55181, 42514.86526042877
Epoch 117601, Training Loss: 46009, Validation Loss: 52120, 39200.038085142136
Epoch 117701, Training Loss: 41066, Validation Loss: 54388, 28400.177388058382
Epoch 117801, Training Loss: 42585, Validation Loss: 57416, 41057.97607321233
Epoch 117901, Training Loss: 44790, Validation Loss: 54603, 34124.326266273274
Epoch 118001, Training Loss: 42297, Validation Loss: 52829, 40856.42867242422
Epoch 118101, Training Loss: 45395, Validation Loss: 57350, 50318.52888535891
Epoch 118201, Training Loss: 46973, Validation Loss: 53542, 42242.863880786375
Epoch 118301, Training Loss: 44940, Validation Loss: 53077, 39188.59472998482
Epoch 118401, Training Loss: 46694, Validation Loss: 55556, 47971.59083045466
Epoch 118501, Training Loss: 47717, Validation Loss: 52316, 35169.54457535642
Epoch 118601, Training Loss: 40643, Validation Loss: 52799, 39081.15380326376
Epoch 118701, Training Loss: 43345, Validation Loss: 55721, 

Epoch 128001, Training Loss: 44348, Validation Loss: 54044, 46972.27189434031
Epoch 128101, Training Loss: 43833, Validation Loss: 54475, 39396.848658995434
Epoch 128201, Training Loss: 45995, Validation Loss: 56759, 38675.09415939096
Epoch 128301, Training Loss: 45890, Validation Loss: 55555, 42487.36900317236
Epoch 128401, Training Loss: 43649, Validation Loss: 54818, 56041.23241057547
Epoch 128501, Training Loss: 43501, Validation Loss: 54350, 39534.75226192679
Epoch 128601, Training Loss: 43050, Validation Loss: 54359, 28071.700795595145
Epoch 128701, Training Loss: 42298, Validation Loss: 53613, 59746.57970099775


In [ ]:
import matplotlib.pyplot as plt

train_losses_sampled = train_losses[::10000]  # Select every 1000th value
val_losses_sampled = val_losses[::10000]      # Select every 1000th value

# Generate corresponding epoch numbers, assuming epochs_suc is your list of epoch numbers
epochs_sampled = epochs_suc[::10000]

plt.style.use("ggplot")
plt.title("Overfitted model evaluation")


# Use sampled data for plotting
plt.plot(epochs_sampled, train_losses_sampled, label='Training')
plt.plot(epochs_sampled, val_losses_sampled, label='Validation')

plt.ylabel("MSE")
plt.xlabel("Epoch")
plt.yscale('log')
plt.xticks(
    range(1, epochs_sampled[-1], int(epochs_sampled[-1] / 8)),
    range(1, epochs_sampled[-1], int(epochs_sampled[-1] / 8)),
    rotation = 25
)
plt.legend()
plt.show()
# plt.savefig("../visualizations/overfit_model_evaluation_full_dataset.png", dpi=800)

# Saving. Good results

In [ ]:
# class TabularFFNNSimple(nn.Module):
#     def __init__(self, input_size, output_size, dropout_prob=0.4):
#         super(TabularFFNNSimple, self).__init__()
#         hidden_size = 48
#         self.ffnn = nn.Sequential(
#             nn.Linear(input_size, hidden_size),
#             nn.ReLU(),
# #             nn.BatchNorm1d(hidden_size),
# #             nn.Dropout(0.5),
#             nn.Linear(hidden_size, hidden_size),
#             nn.ReLU(),
# #             nn.Dropout(0.5),
#             nn.Linear(hidden_size, output_size)
#         )
        
#         for m in self.ffnn:
#             if isinstance(m, nn.Linear):
#                 init.xavier_uniform_(m.weight)
#                 m.bias.data.fill_(0)

#     def forward(self, x):
#         x = x.float()
#         # print(x)
#         x = x.view(x.size(0), -1)
#         x = self.ffnn(x)
#         return x
    
# # Split the data into features and target
# X = data.drop('price', axis=1)
# y = data['price']

# # Standardize the features
# device = torch.device("cpu")
# # Convert to PyTorch tensors
# X_tensor = torch.tensor(X.to_numpy(), dtype=torch.float32, device = device)
# y_tensor = torch.tensor(y.values, dtype=torch.float32, device = device)


# # Split the data into training and combined validation and testing sets
# X_train, X_val_test, y_train, y_val_test = train_test_split(X_tensor, y_tensor,
#                                                             test_size=0.4, random_state=42)

# # Split the combined validation and testing sets
# X_val, X_test, y_val, y_test = train_test_split(X_val_test, y_val_test, test_size=0.5, random_state=42)

# # Create DataLoader for training, validation, and testing
# train_data = TensorDataset(X_train, y_train)
# val_data = TensorDataset(X_val, y_val)
# test_data = TensorDataset(X_test, y_test)

# batch_size = 256
# train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
# val_loader = DataLoader(val_data, batch_size=batch_size, shuffle=False)
# test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=False)

# # Check if the dimensions match the expected input size for the model
# input_size = X_train.shape[1]

# # Output
# # input_size, train_loader, test_loader

# model = TabularFFNNSimple(
#     input_size = input_size,
#     output_size = 1
# )
# model.to(device)

# num_epochs = 300000
# train_losses = []
# val_losses = []
# epochs_suc = [] # to have a reference to it
# grad_norms = []

# def get_gradient_norm(model):
#     total_norm = 0
#     for p in model.parameters():
#         if p.grad is not None:
#             param_norm = p.grad.data.norm(2)
#             total_norm += param_norm.item() ** 2
#     total_norm = total_norm ** 0.5
#     return total_norm

# optimizer = optim.Adam(
#     model.parameters(), 
#     lr=9e-3,
#     weight_decay=1e-4
# )
# criterion = torch.nn.MSELoss()
# criterion_abs = torch.nn.L1Loss()
# criterion = criterion_abs

# scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
#     optimizer, 
#     mode='min', 
#     factor=0.999999, 
#     patience=10, 
#     verbose=True
# )

# for epoch in range(num_epochs):
#     # Training
#     model.train()  # Set the model to training mode
#     running_loss = 0.0
#     l1_losses = []
#     grad_norm = 0
#     for tuple_ in train_loader:
#         datas, prices = tuple_
#         optimizer.zero_grad()
#         outputs = model(datas)
#         prices_viewed = prices.view(-1, 1).float()
#         loss = criterion(outputs, prices_viewed)
#         loss.backward()
#         grad_norm += get_gradient_norm(model)
#         optimizer.step()

#         running_loss += loss.item()
        
#     grad_norms.append(grad_norm / len(train_loader))
#     train_losses.append(running_loss / len(train_loader))  # Average loss for this epoch

#     # Validation
#     model.eval()  # Set the model to evaluation mode
#     val_loss = 0.0
#     with torch.no_grad():  # Disable gradient calculation
#         for tuple_ in val_loader:
#             datas, prices = tuple_
#             outputs = model(datas)  # Forward pass
#             prices_viewed = prices.view(-1, 1).float()
#             loss = criterion(outputs, prices_viewed)  # Compute loss
#             val_loss += loss.item()  # Accumulate the loss
#             l1_losses.append(criterion_abs(outputs, prices_viewed))

#     val_losses.append(val_loss / len(val_loader))  # Average loss for this epoch
#     l1_mean_loss = sum(l1_losses) / len(l1_losses)
#     # Print epoch's summary
#     epochs_suc.append(epoch)
#     scheduler.step(val_losses[-1])
#     if epoch % 100 == 0:
#         tl = f"Training Loss: {int(train_losses[-1])}"
#         vl = f"Validation Loss: {int(val_losses[-1])}"
#         l1 = f"L1: {int(l1_mean_loss)}"
#         dl = f'Epoch {epoch+1}, {tl}, {vl}, {grad_norms[-1]}'
#         print(dl)